# Introdcution

Have you ever wondered how much a captivating picture can influence your decision to adopt or facilitate the adoption of an animal? In the world of the animal welfare, where countless of stray animals are suffering on the streets or in shelters every day, the power of a single image extends beyond aesthetics – it serves as a potential lifeline. With the rise of data science, we aim to help by predicting the appealingness of an image through the "Pawpularity" project to hopefully increase the chances of finding caring homes for these rescue animals.

PetFinder.my is Malaysia’s leading animal welfare platform and it collaborates closely with animal lovers, media, corporations, and global organizations to improve animal welfare. Currently it analyzes cuteness and other factors to rank animals images, which is still in an experimental stage with rooms for improvements. By analyzing both raw images and the metadata, we seek to enhance PetFinder.my's existing approach and provide actionable insights to increase adoption rates. Through this endeavor, we strive to not only improve the fortunes of individual animals but also contribute to the broader cause of animal welfare worldwide.


In [1]:
#Import libraries
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import metrics
from sklearn.model_selection import train_test_split

import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning) #used to supress the tf version warning. 

# FILL IN CODE HERE #
IMAGE_PATH = "./Working/train/"
MEGA_FILE = "./Working/train.csv"

In [2]:
#Import images & labels
def load_data(path_to_images, path_to_label):
    '''Load 2D images and their corresponding labels
    Parameters:
    path_to_data (str): This is the path to data
    
    Returns:
    images (np.ndarray): A numpy array of shape (N, 64, 64, 3)
    labels (np.ndarray): A numpy array of shape (N)
    '''
    #Initialize images and labels as empty list
    images, labels = [], []


    for item in os.listdir(path_to_images):

        #Load image and convert image to array
        #Resize images to 64x64x3
        image = img_to_array(load_img(os.path.join(path_to_images, item)))
        resized_image = tf.image.resize(image, [128, 128])

        #append image and label to the final respective list
        images.append(resized_image)

    mega_file = pd.read_csv(path_to_label)
    labels = mega_file["Pawpularity"]

    #Convert list to numpy array
    images, labels = np.array(images), np.array(labels)

    return images, labels

In [3]:
#Load the images and labels
images, labels = load_data(IMAGE_PATH, MEGA_FILE)
print("Shape of images:", images.shape)
print("Shape of labels:", labels.shape)

Shape of images: (9912, 128, 128, 3)
Shape of labels: (9912,)


In [4]:
## Use a 80/20 train/test split.
images_train, images_test, labels_train, labels_test = train_test_split(images, labels, test_size=0.2, random_state=100)    

images_train = images_train / 255
images_test = images_test / 255

print('train images shape:', images_train.shape)
print('train labels shape:', labels_train.shape)
print('test images shape:', images_test.shape)
print('test labels shape:', labels_test.shape)

train images shape: (7929, 128, 128, 3)
train labels shape: (7929,)
test images shape: (1983, 128, 128, 3)
test labels shape: (1983,)


In [5]:
# Apply random shufflying to training examples.
np.random.seed(100)
indices = np.arange(images_train.shape[0])
shuffled_indices = np.random.permutation(indices)
images_train = images_train[shuffled_indices]
labels_train = labels_train[shuffled_indices]

In [6]:
#Set up baseline model
def predict_baseline(X):
    
    # Calculate the average score for all labels
    avg_score = labels_train.mean()
    
    # Get number of examples in the training set
    m = X.shape[0]
    
    # Generate predictions
    Y = np.dot(np.ones(m), avg_score)
    
    return Y

def RMSE(X, Y):
    
    # Generate predictions of baseline on the train data
    Y_predict = predict_baseline(X)

    # Calculate the difference between predictions and actual prices
    diff = Y_predict - Y

    # Compute the RMSE
    m = X.shape[0]
    RMSE =  np.sqrt(np.sum(np.square(diff)) / m)
    
    return RMSE


print("Predictions for train data baseline output: \n", predict_baseline(images_train), "\n")
print("Predictions for testing set baseline output: \n", predict_baseline(images_test), "\n")

RMSE_baseline_train = RMSE(images_train, labels_train)
RMSE_baseline_test = RMSE(images_test, labels_test)
print("RMSE of the baseline on the train data: \n", RMSE_baseline_train, "\n")
print("RMSE of the baseline on the test data: \n", RMSE_baseline_test)

Predictions for train data baseline output: 
 [37.95913734 37.95913734 37.95913734 ... 37.95913734 37.95913734
 37.95913734] 

Predictions for testing set baseline output: 
 [37.95913734 37.95913734 37.95913734 ... 37.95913734 37.95913734
 37.95913734] 

RMSE of the baseline on the train data: 
 20.599719353543417 

RMSE of the baseline on the test data: 
 20.55663146623801


In [7]:
# Define the CNN model
def build_cnn():

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=64, kernel_size=5, activation="relu", padding="same"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Conv2D(
        filters=128, kernel_size=5, activation="relu", padding="same"))   
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Conv2D(
        filters=256, kernel_size=5, activation="relu", padding="same"))   
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=512, activation="relu"))
    model.add(tf.keras.layers.Dense(units=1, activation="linear"))

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
                  loss="mean_squared_error",
                  metrics=["accuracy"])

    return model

CNN = build_cnn()

# use early stopping
early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=3)

# Train the model
history = CNN.fit(images_train,
                  labels_train,
                  epochs=20,
                  validation_split=0.1,
                  verbose=0,
                  callbacks=[early_stopping_callback])

train_loss, train_metric = CNN.evaluate(images_train, labels_train)
train_RMSE = np.sqrt(train_loss)
print("Final RMSE on train data is", train_RMSE)

test_loss, test_metric = CNN.evaluate(images_test, labels_test)
test_RMSE = np.sqrt(test_loss)
print("Final RMSE on test data is", test_RMSE)

62/62 [==============================] - 16s 256ms/step - loss: 422.8158 - accuracy: 0.0000e+00
Final RMSE on test data is 20.562484416549946


248/248 [==============================] - 64s 257ms/step - loss: 424.9432 - accuracy: 5.0448e-04
Final RMSE on train data is 20.61415138453891
